# Time Series Prediction with Wheat Prices

This is a short time series example with the Kaggle Dataset [Cerial Prices Changes Within Last 30 Years](https://www.kaggle.com/datasets/timmofeyy/-cerial-prices-changes-within-last-30-years?select=rice_wheat_corn_prices.csv).

## Introduction to the Data

As described in **Kaggle**, the variables are as follows:

* **Year :** A year starting from 1992 ending with 2022
* **Month :** A month of the year
* **Price_wheat_ton :** Wheat price per ton in the definite month and year
* **Price_wheat_ton_infl :** Modern price per ton of wheat
* **Price_rice_ton :** Price price per ton in the definite month and year
* **Price_rice_ton_infl :** Modern rice per ton of rice
* **Price_corn_ton** : Corn price per ton in the definite month and year
* **Price_corn_ton_inf :** Modern price per ton of corn
* **Inflation_rate :** Prices are generally continuous. For example this year price is a 1 dollar for a corn, but next year maybe 2 dollar.

However, we will only use the variables **Year**, **Month**, and **Price_wheat_ton** solely for this project.

## Our Goals and Analysis for this Data

So since we're only analyzing the wheat price per ton for that specific month and year, we wanted to see the trend for the price of wheat, compare the predictability of different time series models that we're going to create, and the pros and cons of each model. Specifically, our goals are:

* Analyze the pricing trend and determine any seasonal effects 
* Use prior years' price to predict the price of wheat using a series of models (linear regression models, splines, and neural networks, etc) for **one year- 2021**.
* Analyze the pros anc cons of each model in predicting 2021 price data.


**We will now setup and load the packages we need along with the data.**

In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from scipy.signal import periodogram
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.graphics.tsaplots import plot_pacf


# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 4))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)
%config InlineBackend.figure_format = 'retina'


def lagplot(x, y=None, lag=1, standardize=False, ax=None, **kwargs):
    from matplotlib.offsetbox import AnchoredText
    x_ = x.shift(lag)
    if standardize:
        x_ = (x_ - x_.mean()) / x_.std()
    if y is not None:
        y_ = (y - y.mean()) / y.std() if standardize else y
    else:
        y_ = x
    corr = y_.corr(x_)
    if ax is None:
        fig, ax = plt.subplots()
    scatter_kws = dict(
        alpha=0.75,
        s=3,
    )
    line_kws = dict(color='C3', )
    ax = sns.regplot(x=x_,
                     y=y_,
                     scatter_kws=scatter_kws,
                     line_kws=line_kws,
                     lowess=True,
                     ax=ax,
                     **kwargs)
    at = AnchoredText(
        f"{corr:.2f}",
        prop=dict(size="large"),
        frameon=True,
        loc="upper left",
    )
    at.patch.set_boxstyle("square, pad=0.0")
    ax.add_artist(at)
    ax.set(title=f"Lag {lag}", xlabel=x_.name, ylabel=y_.name)
    return ax


def plot_lags(x, y=None, lags=6, nrows=1, lagplot_kwargs={}, **kwargs):
    import math
    kwargs.setdefault('nrows', nrows)
    kwargs.setdefault('ncols', math.ceil(lags / nrows))
    kwargs.setdefault('figsize', (kwargs['ncols'] * 2, nrows * 2 + 0.5))
    fig, axs = plt.subplots(sharex=True, sharey=True, squeeze=False, **kwargs)
    for ax, k in zip(fig.get_axes(), range(kwargs['nrows'] * kwargs['ncols'])):
        if k + 1 <= lags:
            ax = lagplot(x, y, lag=k + 1, ax=ax, **lagplot_kwargs)
            ax.set_title(f"Lag {k + 1}", fontdict=dict(fontsize=14))
            ax.set(xlabel="", ylabel="")
        else:
            ax.axis('off')
    plt.setp(axs[-1, :], xlabel=x.name)
    plt.setp(axs[:, 0], ylabel=y.name if y is not None else x.name)
    fig.tight_layout(w_pad=0.1, h_pad=0.1)
    return fig


In [2]:
df = pd.read_csv('../input/-cerial-prices-changes-within-last-30-years/rice_wheat_corn_prices.csv')
df

## Exploratory Data Analysis & Data Cleaning

We wanted to get the **Month** and **Year** to be a single date variable. Additionally, we wanted to check for anomalies that might affect the model that we build. We check for the summary statistics as follows:

In [4]:
df.describe()

THe price of wheat seems pretty reasonable, except that the maximum price may be a bit **too** high. It is almost twice as much as the mean price and the 75th percentile price. However, we would not do anything to correct or remove the highest price, as it might be essential to our analysis later on.

### Converting the Year and Month into a Date Format

Now, we will have a variable **YearMonth**, that merge the two date data columns (**Year** and **Month**) into one.

In [3]:
from time import strptime
df['Month']

df['YearMonth'] = pd.date_range('1992-02','2022-01',freq='MS').strftime('%Y-%m')
df.head()



### Taking care of Null Values

Now that we have that taken care of, we would need to further analyze and see if there were any null values, and take care of it before we run any models.

In [6]:
df1 = df[df.isna().any(axis=1)]
df1

We see that the last row and the most recent **Wheat Price in Tons** data point has a null value. Instead of taking care of the anomaly by having the null value of the **Wheat Price in Tons** of January 2022 be **zero**, we will drop this entry, since it won't create a break in time with the lack of this variable, and won't affect our model predictions. We are only targeting the predictability of the year **2021** with the data from previous years dating back to **1992**.

Along with dropping the row with the null value, we are going to add another variable, **YMD**, that gives the actual beginning date of each month and year in case the model we are working with only accepts a certain format.

In [4]:
df=pd.DataFrame.dropna(df)

df['YMD'] = pd.date_range('1992-02','2021-12',freq='MS')

## Detecting Trends

After finishing with data cleaning, we should move to see some of the data trends. With a **moving average plot**, we can smooth out any short-term fluctuations in the series so that only long-term changes remain.

When computing a moving average of a time series, we take the average values within a sliding window of a defined width. We will start with a 6-month moving average graph, followed by a 1-year moving average.

In [14]:
test1 = df.Price_wheat_ton.rolling(
    window=6,       # 6 month window, or a 6 month moving average
    center=True,      # puts the average at the center of the window
).mean()              # compute the mean (could also do median, std, min, max, ...)

test1.plot()

df.Price_wheat_ton.plot(title="Wheat Prices in Tons - 6-Month Moving Average", xlabel='Number of Months from Feb. 1992', ylabel='Price' )

In [10]:
test2 = df.Price_wheat_ton.rolling(
    window=12,       # 6 month window, or a 6 month moving average
    center=True,      # puts the average at the center of the window
).mean()              # compute the mean (could also do median, std, min, max, ...)

test2.plot()

df.Price_wheat_ton.plot(title="Wheat Prices in Tons - 12-Month Moving Average", xlabel='Number of Months from Feb. 1992', ylabel='Price')

We can see that the trend has no clear pattern and fluctuates similar to a polynomial regression, meaning that we are working with non-stationary data. 

What happens if we do a **50-month** moving average? We should test this out!

In [16]:
test3 = df.Price_wheat_ton.rolling(
    window=50,       # 6 month window, or a 6 month moving average
    center=True,      # puts the average at the center of the window
).mean()              # compute the mean (could also do median, std, min, max, ...)

test3.plot()

df.Price_wheat_ton.plot(title="Wheat Prices in Tons - 50-Month Moving Average", xlabel='Number of Months from Feb. 1992', ylabel='Price')

## What we can determine from the trend

This has a clearer polynomial linear regression trend than was presented compared to when we do a 6-month or 12-month average. Since we've identified the shape of the trend, we can attempt to model it using a time-step feature. 

We can determine that the data that we are working with is non-stationary data, meaning that the statistical measures, such as the mean, standard deviation, and auto correlation show a decreasing or increasing trend over time. In this case, it is an increasing trend over time.

## Detecting Seasonality

We can also choose to decompose different parts of the time series with **seasonal_decompose** from the **statsmodels** package. As you can see, from the graphical example below, we analyze that:
* The trend is fluctuating, showing a series of increase and decrease over time, but moving towards increase.
* For the seasonal portion of the data, we see more of a yearly pattern.
* As for the residual portion of the data, this isthe portion that cannot be explained by the trend or seasons. It is the noise portion of the dataset.


In [5]:
df1 = df[['Price_wheat_ton','YMD']]
df1.set_index('YMD',inplace=True)
from statsmodels.tsa.seasonal import seasonal_decompose
trend_detect=seasonal_decompose(df1['Price_wheat_ton'])
trend_detect.plot();

We shall add another variable, **Time**, which dates the number of months from the beginning of the series (February 1992), in case of further analysis.

In [6]:
df['Time'] = np.arange(len(df.index))
# df['Lag_1_W'] = df['Price_wheat_ton'].shift(1)

# Model Making

We can choose to make the data stationary for the machine to better detect the trend and the pattern. However, I have decided to use models that were geared towards non-stationary data. Since we are using non-stationary data, we wouldn't use some of the models geared toward stationary data with a constant mean and variance, such as the **ARIMA** model. 

## Simple First Order Linear Regression
We fit a simple first order linear regression in the data. You can tell that it does not do well, since it has a polynomial linear trend.

In [24]:
fig, ax = plt.subplots()
ax.plot('Time', 'Price_wheat_ton', data=df, color='0.75')
ax = sns.regplot(x='Time', y='Price_wheat_ton', data=df, ci=None, scatter_kws=dict(color='0.25'))
ax.set_title('Time Plot of Wheat Price per Ton');

## Using Deterministic Process and Trends

**Deterministic trends** are constant increases in the mean of the series over time, though the variable may fluctuate above or below its trend line randomly. It is a non-stationary process, since it violates the stationarity assumptions due to its variable being unable to revert toward a fixed mean after any shock.

We will use **DeterministicProcess** to create features for dates. It is a technical term for a time series that is non-random or completely determined. Features derived from the time index will generally be deterministic.

In [8]:
from statsmodels.tsa.deterministic import DeterministicProcess
y = df["Price_wheat_ton"]  # the target

dp = DeterministicProcess(
    index=df.YearMonth,  # dates from the training data
    constant=True,       # dummy feature for the bias (y_intercept)
    order=5,             # the time dummy (trend)
    drop=True,           # drop terms if necessary to avoid collinearity
)
# `in_sample` creates features for the dates given in the `index` argument
X = dp.in_sample()

X.head()

## Polynomial Linear Regression 

After our **DeterministicProcess** feature creation, we shall be running a polynomial linear regression with the feature that we have.

In [9]:
from sklearn.linear_model import LinearRegression

y = df["Price_wheat_ton"]  # the target
# The intercept is the same as the `const` feature from
# DeterministicProcess. LinearRegression behaves badly with duplicated
# features, so we need to be sure to exclude it here.
model = LinearRegression()
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)

We will use the **root-mean-square error (RMSE)** to measure the differences between values (sample or population values) predicted by a model or an estimator and the values observed from the dataset.

In [10]:
from sklearn.metrics import mean_squared_error

rmse=(mean_squared_error(df['Price_wheat_ton'],y_pred))
print(rmse)

### Linear Regression with Deterministic Process (Order of 5)

As you can see, when fitting a 5th order linear regression model into the wheat price data, this regression does seem like a good fit from a graphical standpoint. It looks similar to the **50-Month Moving Average Plot** that we created in the trend portion of the project.

In [28]:

ax = df.Price_wheat_ton.plot(style=".", color="0.5", title="Wheat Price - Linear Trend")
_ = y_pred.plot(ax=ax, linewidth=3, label="Trend")

So what happens if you tried predicting the next set of prices?

In [29]:
X = dp.out_of_sample(steps=12)

y_fore = pd.Series(model.predict(X), index=X.index)

ax = df.Price_wheat_ton.plot(title="Wheat Price - Linear Trend", **plot_params)
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend")
ax = y_fore.plot(ax=ax, linewidth=3, label="Trend Forecast", color="C3")
_ = ax.legend()

Now, the trend forecast might go a bit too high, maybe higher than the maximum historical wheat price per ton. The prediction is likely to be too steep of an increase from what the actual might look like. Although it was able to predict the direction of the trend, the actual might not have a pricing increase as fast as predicted. 

### Linear Regression with Deterministic Process (Order of 7)

We are going to look at what happens when fitting a higher order of linear regression model (a 7th order linear regression).

In [30]:
dp = DeterministicProcess(
    index=df.YearMonth,  # dates from the training data
    constant=True,       # dummy feature for the bias (y_intercept)
    order=7,             # the time dummy (trend)
    drop=True,           # drop terms if necessary to avoid collinearity
)
# `in_sample` creates features for the dates given in the `index` argument
X = dp.in_sample()

y = df["Price_wheat_ton"]  # the target
# The intercept is the same as the `const` feature from
# DeterministicProcess. LinearRegression behaves badly with duplicated
# features, so we need to be sure to exclude it here.
model = LinearRegression()
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)

ax = df.Price_wheat_ton.plot(style=".", color="0.5", title="Wheat Price - Linear Trend")
_ = y_pred.plot(ax=ax, linewidth=3, label="Trend")

In [15]:
from sklearn.metrics import mean_squared_error

rmse=(mean_squared_error(df['Price_wheat_ton'],y_pred))
print(rmse)

Now, when fitting a 7th order linear regression model into the wheat price data, this regression is not as much of a good fit as compared to the 5th order linear regression model, as shown that the RMSE is higher in the 7th order regression model. Some of the price fluctuations that we see between 2005 and 2017 isn't as detailed as we see with the 5th order polynomial regression. 

However, we should see how the 7th order polynomial regression predicts later time points.

In [31]:
X = dp.out_of_sample(steps=12)

y_fore = pd.Series(model.predict(X), index=X.index)

ax = df.Price_wheat_ton.plot(title="Wheat Price - Linear Trend", **plot_params)
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend")
ax = y_fore.plot(ax=ax, linewidth=3, label="Trend Forecast", color="C3")
_ = ax.legend()

We can see that the 7th order polynomial linear regression also predicts a price increase, though it is not as steep and exponential as compared to the 5th order polynomial regression. The *direction of the trend* prediction is the same, but the **rate of increase differs**, and we can agree on that the 7th order polynomial linear regression shows a more reasonable prediction, but the 5th order polynomial linear regression gives a better model of the historical data.

## Linear Regression with Fourier Features

We wanted to choose seasonal features to see if it will add better predictive capability to our model. If we treat a seasonal period as a categorical feature and apply one-hot encoding, would it give a better model to our polynomial linear regression?

We will do a **periodogram** to see how many Fourier pairs we should actually include in our features, since it tells us the strength of the frequencies in a time series. 

In [44]:
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'


# annotations: https://stackoverflow.com/a/49238256/5769929
def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax


def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax



In [45]:
plot_periodogram(df.Price_wheat_ton);

### Periodogram Analysis

From left to right, the periodogram drops off after Annually and Semiannually. 
It shows a strong annual season, which means that we can model the annual season with Fourier features. From right to left, the periodogram falls off around Monthly (12), so let's use 12 Fourier pairs.


**Fourier features** try to capture the overall shape of the seasonal curve with just a few features instead of creating a feature for each date. By using Fourier features, we can capture frequencies within a season and include in our training data the periodic curves of similar frequencies as the season we are trying to model. 

We'll create our seasonal features using DeterministicProcess. 

In [22]:
from statsmodels.tsa.deterministic import CalendarFourier
y = df.Price_wheat_ton

fourier = CalendarFourier(freq="A", order=12)  # 10 sin/cos pairs for "A"nnual seasonality

dp = DeterministicProcess(
    index=df.YMD,
    constant=True,               # dummy feature for bias (y-intercept)
    order=5,                     # trend (order 1 means linear)
    seasonal=True,               
    additional_terms=[fourier],  # annual seasonality (fourier)
    drop=True,                   # drop terms to avoid collinearity
)

X = dp.in_sample() 

In [23]:
model = LinearRegression().fit(X, y)
y_pred = pd.Series(
    model.predict(X),
    index=X.index,
    name='Fitted',
)

y_pred = pd.Series(model.predict(X), index=X.index)
ax = df1.plot(color='0.25', style='.', title="Wheat Price - Seasonal Forecast with Fourier")
ax = y_pred.plot( label="Seasonal")
ax.legend();

In [24]:
rmse=(mean_squared_error(df['Price_wheat_ton'],y_pred))
print(rmse)

From the graph and the root mean square error above, using the Fourier feature gives better predictability as compared to the polynomial linear regression that we ran previously, since it took seasonality into account. 


### Predictive capability of using Fourier Features

We will run a test to see how the model predicts the last 12 months of data using the prior historical data from before 2021. 

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=12, shuffle=False)

# Fit and predict
model = LinearRegression()  # `fit_intercept=True` since we didn't use DeterministicProcess
model.fit(X_train, y_train)
y_pred = pd.Series(model.predict(X_train), index=y_train.index)
y_fore = pd.Series(model.predict(X_test), index=y_test.index)

In [27]:
ax = y_train.plot(**plot_params)
ax = y_test.plot(**plot_params)
ax = y_pred.plot(ax=ax, title="Wheat Prices in Tons over Time", xlabel= "Number of Months after Feb 1992", ylabel="Price")
_ = y_fore.plot(ax=ax, color='C3')

We are certain that, with **Red** being the predicted result, and **Blue** as the training model, and **Black** are the actual prices in 2021, this is a good predictive model compared to the previous polynomial regression model that did not use Fourier features. 

## Using Splines Instead

Now with splines, or **multivariate adaptive regression splines (MARS)**, we can form a non-parametric linear regression model that automatically models nonlinearities and interactions between variables. Let's see how good it models the price of wheat over time.

In [28]:
from pyearth import Earth

# Target and features are the same as before
y = df["Price_wheat_ton"].copy()
dp = DeterministicProcess(index=y.index, order=1)
X = dp.in_sample()

# Fit a MARS model with `Earth`
model = Earth()
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)

ax = y.plot(**plot_params, title="Price of Wheat over time from 1992 (in months)", ylabel="Price")
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend")

In [29]:
rmse=(mean_squared_error(df['Price_wheat_ton'],y_pred))
print(rmse)

We can see from the RMSE that MARS models better than polynomial regression. Will it have better predictive capability using pre-2021 data to determine the data for 2021?

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=12, shuffle=False)

# Fit and predict
model = Earth()  # `fit_intercept=True` since we didn't use DeterministicProcess
model.fit(X_train, y_train)
y_pred = pd.Series(model.predict(X_train), index=y_train.index)
y_fore = pd.Series(model.predict(X_test), index=y_test.index)

ax = y_train.plot(**plot_params)
ax = y_test.plot(**plot_params)
ax = y_pred.plot(ax=ax, title="Wheat Prices in Tons over Time", xlabel= "Number of Months after Feb 1992", ylabel="Price")
_ = y_fore.plot(ax=ax, color='C3')

We can see that, compared to the polynomial regression model (5th order) using the Fourier Feature, the MARS model **have better modeling capability**, but **doesn't have a better predictive capability** in terms of using dataset from 1992 to 2020 to predict the actual data in 2021.

## Multistep Forecasting with Lags

With a **multioutput model**, we can produce a model that gives multiple outputs naturally. There are a number of strategies for producing the multiple target steps required for a forecast, but we will focus on using the direct strategy and a combination of direct and recursive strategies.

As described from [Forecasting with Machine Learning](https://www.kaggle.com/code/ryanholbrook/forecasting-with-machine-learning), the definition of each strategy is as follows:

* **Direct strategy**: Train a separate model for each step in the horizon: one model forecasts 1-step ahead, another 2-steps ahead, and so on. Forecasting 1-step ahead is a different problem than 2-steps ahead (and so on). However, training lots of models can be computationally expensive.

* **Recursive strategy**: Train a single one-step model and use its forecasts to update the lag features for the next step. With the recursive method, we feed a model's 1-step forecast back in to that same model to use as a lag feature for the next forecasting step. We only need to train one model, but since errors will propagate from step to step, forecasts can be inaccurate for long horizons.

* **DirRec strategy**: A combination of the direct and recursive strategies: train a model for each step and use forecasts from previous steps as new lag features. Step by step, each model gets an additional lag input. Since each model always has an up-to-date set of lag features, the DirRec strategy can capture serial dependence better than Direct, but it can also suffer from error propagation like Recursive.

### Start by making Lag Features

So we will start by making lag features. A lag 1 feature shifts the time series forward 1 step, which means you could forecast 1 step into the future, but not 2 steps.

In [11]:
from pathlib import Path
import ipywidgets as widgets
from learntools.time_series.style import *  # plot style settings
from learntools.time_series.utils import (create_multistep_example,
                                          load_multistep_data,
                                          make_lags,
                                          make_multistep_target,
                                          plot_multistep)


y = df.loc[:, 'Price_wheat_ton']

#  Make 6 lag features
X = make_lags(y, lags=6).dropna()

# Make multistep target
y = make_multistep_target(y, steps=12).dropna()

y, X = y.align(X, join='inner', axis=0)

### Direct Strategy

We will use a multioutput Direct strategy to test the model fit.

In [14]:
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import RegressorChain
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


# Create splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

model = MultiOutputRegressor(XGBRegressor())
model.fit(X_train, y_train)

y_fit = pd.DataFrame(model.predict(X_train), index=X_train.index, columns=y.columns)
y_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)

train_rmse = mean_squared_error(y_train, y_fit, squared=False)
test_rmse = mean_squared_error(y_test, y_pred, squared=False)
print((f"Train RMSE: {train_rmse:.2f}\n" f"Test RMSE: {test_rmse:.2f}"))


### DirRec Strategy

We will use a multioutput DirRec strategy to test the model fit.

In [15]:


model = RegressorChain(XGBRegressor())
model.fit(X_train, y_train)

y_fit = pd.DataFrame(model.predict(X_train), index=X_train.index, columns=y.columns)
y_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)

train_rmse = mean_squared_error(y_train, y_fit, squared=False)
test_rmse = mean_squared_error(y_test, y_pred, squared=False)
print((f"Train RMSE: {train_rmse:.2f}\n" f"Test RMSE: {test_rmse:.2f}"))



### Direct and DirRec Strategy Comparison

We can see, from the RMSE analysis above, that using the Direct strategy to model Wheat Prices is more accurate and closer to the actual Wheat Price than using the DirRec strategy.

## Neural Networks

We wanted to try another way- neural networks, to see if it is possible to train the computer to assign different weights and predict better than linear regression models, splines, or multistep direct recursive model.

### Scaling

We will scale the training set (1992-2020 data) and the testing set (2021 data) from 0 to 1 to better fit in the neural network.

In [16]:
train= df1.iloc[:-12]
test= df1.iloc[-12:]

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

#scale the data from 0 to 1

scaler.fit(train)
scaled_train= scaler.transform(train)
scaled_test= scaler.transform(test)

### Generators and Features

With neural network, we would be defining the generators and features of the model. The number of features that we have is only 1, which comes from the time variable that we have. However, we can set the number of input for the generator matrix.

In [51]:
from keras.preprocessing.sequence import TimeseriesGenerator

#define the generators and features
input_n= 6 #6 months
features= 1
generator= TimeseriesGenerator(scaled_train, scaled_train, length=input_n, batch_size=1)

### Making the Neural Networks model with layers

We are going to use the **sequential neural networks** with a **LSTM** artificial recurrent neural network (RNN) architecture. The number of layers that we are going to have depends on our preference. I set up 100 layers as an experiment. The Dense function tells the machine that we want whatever input that we put into the model to give us that specific number of output. We can see the summary of our model after we set it up.

In [52]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(100,input_shape=(input_n, features)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()


### Running the model multiple times

Now that we have the model, we can run it multiple times, or epochs, to minimize the loss with each learning rate and gradient descent from the neural network.

In [53]:
model.fit(generator, epochs=100, batch_size=1)

### Loss History

When we look at the loss history for neural network, we can see how many epochs we needed to run until the loss is actually minimized to a number that cannot be minimized further when running gradient descent for our neural network model.

In [54]:
PerEpochLoss=model.history.history['loss']
plt.plot(range(len(PerEpochLoss)),PerEpochLoss)

As you can see, the loss reached a minimum after around 10-13 epochs. We can stop the gradient descent at around 13 epochs.

### Fitting the Training set

So we wanted to test to see whether we can take the last n value of the training set to predict the first value of the test set. We are going to compare the predicted value with the actual value of the test set.

In [55]:
#taking the last 6 values of the training set and predict the first value of the test set
lasttrainbatch=scaled_train[-6:]
lasttrainbatch=lasttrainbatch.reshape((1,input_n, features))

model.predict(lasttrainbatch)

In [56]:
scaled_test[0]
#pretty close to the actual value!

As you can see, there is only a slight difference between the predicted value and the actual value. We will proceed to predict the 2021 value with our current neural network model now.

In [57]:
test_predictions=[]

first_eval_batch= scaled_train[-input_n:]
current_batch= first_eval_batch.reshape((1,input_n,features))

for i in range(len(test)):
    
    currentprediction=model.predict(current_batch)[0]
    test_predictions.append(currentprediction)
    current_batch=np.append(current_batch[:,1:,:],[[currentprediction]],axis=1)

### Scaling back

After we got our prediction figures, we should scale the figures back to actual wheat prices per ton instead of a figure between 0 and 1.

In [58]:
#transform this back to original scale

test_predictions= scaler.inverse_transform(test_predictions)

test['Predictions']= test_predictions

### Comparison of Prediction figures and Actual figures

Let's compare the prediction value and the actual value of the wheat price.


In [59]:
test.plot()

In [60]:
from sklearn.metrics import mean_squared_error

rmse=(mean_squared_error(test['Price_wheat_ton'],test['Predictions']))
print(rmse)

## What the Neural Network Model Tell Us

As seen in the model above, the neural network model can also predict the upward trend of the wheat price, but it doesn't have as good of a future predictive capability as compared to the Fourier feature polynomial regression model. From the RMSE we've calculated, it is similar to the RMSE we get from linear regression models. 

From what we can interpret, it is possible that models that have better modeling capability of the data (splines and multioutput models) might not have a good predictive capability due to overfitting. Overall, the fifth order Fourier feature polynomial regression model takes a moderate approach of infusing a proper modeling capability with a better predictive capability.